In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microcontroller/completeBhageeraPayload.csv


In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv("/kaggle/input/microcontroller/completeBhageeraPayload.csv")

In [4]:
df

,mlid,ax,ay,az
0,Walking,-0.430176,0.611572,-0.747803
1,Walking,-0.600586,0.668945,-0.920898
2,Walking,-0.390137,0.568115,-0.972168
3,Walking,-1.108887,0.886475,-0.582275
4,Walking,-0.442139,0.665039,-0.820068
...,...,...,...,...
25424,Standing,-0.445557,0.597412,-0.812744
25425,Standing,-0.332031,0.624268,-0.785645
25426,Standing,-0.386475,0.446533,-0.897461
25427,Standing,-0.481689,0.316162,-0.902588


In [5]:
df['mlid'].value_counts()

Rest        8141
Sleep       5753
Walking     5241
Running     3698
Standing    1757
Sitting      839
Name: mlid, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
yle = LabelEncoder()
df['mlid'] = yle.fit_transform(df['mlid'])
df.head()

,mlid,ax,ay,az
0,5,-0.430176,0.611572,-0.747803
1,5,-0.600586,0.668945,-0.920898
2,5,-0.390137,0.568115,-0.972168
3,5,-1.108887,0.886475,-0.582275
4,5,-0.442139,0.665039,-0.820068


In [7]:
df['mlid'].value_counts()

0    8141
3    5753
5    5241
1    3698
4    1757
2     839
Name: mlid, dtype: int64

In [8]:
X = df.drop('mlid',axis=1)
Y=df['mlid']

In [9]:
from imblearn.combine import SMOTETomek

In [10]:
smk = SMOTETomek(random_state=42)

In [11]:
X_res , y_res = smk.fit_resample(X,Y)

In [12]:
y_res.value_counts()

3    8022
4    7980
2    7903
1    7778
5    7627
0    7554
Name: mlid, dtype: int64

In [13]:
# Splitting the data into training and testing set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 0)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.utils import to_categorical

# Assuming you have preprocessed data: X_train, y_train, X_test, y_test

# Convert the labels to one-hot encoded vectors
num_classes = 6  # Number of unique classes
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)


# Define the architecture of the ANN model
model = Sequential()
model.add(Dense(64, activation='relu'))  # input_dim represents the number of input features
# model.add(Dropout(0.1))  # Add a dropout layer with a dropout rate of 0.1
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))  # Add another dropout layer with a dropout rate of 0.2
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model_history = model.fit(X_train, y_train_encoded, epochs=50, batch_size=32)


Epoch 1/50
1172/1172 [==============================] - 10s 4ms/step - loss: 0.7212 - accuracy: 0.7436
Epoch 2/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.5571 - accuracy: 0.8025
Epoch 3/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.5328 - accuracy: 0.8078
Epoch 4/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.5198 - accuracy: 0.8102
Epoch 5/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.5079 - accuracy: 0.8153
Epoch 6/50
1172/1172 [==============================] - 5s 4ms/step - loss: 0.4983 - accuracy: 0.8174
Epoch 7/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.4901 - accuracy: 0.8224
Epoch 8/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.4874 - accuracy: 0.8232
Epoch 9/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.4801 - accuracy: 0.8266
Epoch 10/50
1172/1172 [==============================] - 4s 4ms/step - loss: 0.47

In [15]:
print(model_history.history.keys())

dict_keys(['loss', 'accuracy'])


In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_encoded)


293/293 [==============================] - 1s 3ms/step - loss: 0.4057 - accuracy: 0.8538


In [17]:
from keras.models import save_model

In [18]:
save_model(model,"model.pb")

In [19]:
import tensorflow as tf

# Load the SavedModel
model = tf.saved_model.load('/kaggle/working/model.pb')

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/model.pb')
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the TFLite model
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)